# Monthly wages for females and males
In LOCAL CURRENCY UNITS

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict
import functools
import operator

In [ ]:
# !pip install easymoney
# from easymoney.money import EasyPeasy 

In [ ]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
files = ['MONWAGFE.csv', 'MONWAGMA.csv']
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append((f, df))    

## Let's narrow down to a chosen year

In [ ]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [ ]:
# import US specific data (median dollar earnings per month)
us_data = pd.read_csv("median_US_earnings.csv")
us_data.head()  

In [ ]:
wages_fem = dfs[0][1]  # monthly wages for females
wages_m = dfs[1][1] # monthly wages for males


In [ ]:
wages_fem = wages_fem.iloc[0:85, :]  # rm bottom descrip rows
wages_m = wages_m.iloc[0:85, :]  # rm bottom descrip rows

In [ ]:
# narrow down to countries that DO have data for 2011
wages_fem = wages_fem[~wages_fem['2011'].isna()]
wages_m = wages_m[~wages_m['2011'].isna()]
print(wages_fem.shape, wages_m.shape) # (26,18) -> we have 26 countries

In [ ]:
wages_fem = wages_fem.fillna(axis=0, method='pad')
wages_m = wages_m.fillna(axis=0, method='pad')

In [ ]:
print(wages_fem.isna().sum().sum())
print(wages_m.isna().sum().sum())

In [ ]:
# process us data separartely
us_data2 = us_data.transpose()
us_data2.columns = us_data2.iloc[0,:]
# us_data2['ISO'] = 'USA'
wages_fem_us = pd.DataFrame(us_data2.iloc[1,:])
wages_m_us = pd.DataFrame(us_data2.iloc[2,:])


In [ ]:
wages_m_us.rename(columns={'male': 'value'}, inplace=True)
wages_fem_us.rename(columns={'female': 'value'}, inplace=True)


In [ ]:
wages_fem_us['gender'] = 'female'
wages_fem_us.reset_index(inplace=True)
wages_m_us['gender'] = 'male'
wages_m_us.reset_index(inplace=True)

In [ ]:
wages_us = pd.concat([wages_fem_us, wages_m_us], axis=0)
wages_us['ISO'] = 'USA'
wages_us2 = wages_us[wages_us['year'] != 'ISO']
# wages_us2.drop(labels='index', axis=1, inplace=True)

In [ ]:
# wages_us2.head()

In [ ]:
## combine BOTH genders one dataset for ONE country
""" 
1. Narrow down country
2. Separately for each M / F dataset, melt the dataset
3. concat the melted datasets together
"""
all_countries = pd.DataFrame()
countries = list(wages_fem['ISO'].unique())
countries.remove("MLT")


## Choose a country
for c in countries:
# c = 'AUS'
    f1 = wages_fem[wages_fem['ISO'] == c]
    f2 = f1.melt().iloc[1:, :]
    f2.columns = ['year', 'value']
    f2['gender'] = 'female'
    m1 = wages_m[wages_m['ISO'] == c]
    m2 = m1.melt().iloc[1:, :]
    m2.columns = ['year', 'value']
    m2['gender'] = 'male'
    all = pd.concat([f2, m2], axis=0)
    all2 = all[all['year'] != 'gender']
    all2['ISO'] = c
    # all2.head()
    all_countries = pd.concat([all_countries, all2], axis=0)


In [ ]:
# all_countries.head()
print(all_countries.shape)

In [ ]:
# all_countries.reset_index(inplace=True)
# wages_us2.reset_index(inplace=True)
all_countries2 = pd.concat([all_countries, wages_us2], axis=0)
print(all_countries2.shape)

In [ ]:
## check for missing values
# df.isna().sum().sort_values(ascending=False)
# df.drop(labels='index', axis=1, inplace=True)

In [ ]:
df = all_countries2

In [ ]:
## Map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
country_names_dict = { v: k for k, v in country_names.items()}
def convert_iso_to_name(iso):
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso
# df['country'] = df['ISO']
df['country'] = df['ISO'].apply(convert_iso_to_name)
# df.drop(labels='ISO', axis=1, inplace=True)

In [ ]:
# Convert ISO to currency codes
df['currency'] = df['ISO']
df['currency'] = df['currency'].str.replace("ECU", "USD") # ecuador uses USD
df['currency'] = df['currency'].str.replace("DEU", "EUR") # germany uses the Euro
df['currency'] = df['currency'].str.replace("GTM", "GTQ")
df['currency'] = df['currency'].str.replace("ISL", "ISK")
df['currency'] = df['currency'].str.replace("USA", "USD")
df['currency'] = df['currency'].str.replace("LVA", "EUR")
df['currency'] = df['currency'].str.replace("LUX", "EUR")
df['currency'] = df['currency'].str.replace("MEX", "MXN")
df['currency'] = df['currency'].str.replace("MNG", "MNT")
df['currency'] = df['currency'].str.replace("NZL", "NZD")
df['currency'] = df['currency'].str.replace("NOR", "NOK")
df['currency'] = df['currency'].str.replace("PAK", "PKR")
df['currency'] = df['currency'].str.replace("PHL", "PHP")
df['currency'] = df['currency'].str.replace("QAT", "QAR")
df['currency'] = df['currency'].str.replace("SGP", "SGD")
df['currency'] = df['currency'].str.replace("SVK", "EUR")
df['currency'] = df['currency'].str.replace("ESP", "EUR")
df['currency'] = df['currency'].str.replace("SWE", "SEK") # sweden
df['currency'] = df['currency'].str.replace("SYR", "SYP") # syrian
df['currency'] = df['currency'].str.replace("THA", "THB") 
df['currency'] = df['currency'].str.replace("UKR", "UAH")
df['currency'] = df['currency'].str.replace("GBR", "GBP") 
df['currency'] = df['currency'].str.replace("URY", "UYU") 


In [ ]:
# ## convert local currency to USD
# ## for D3 chart so the axes are consistent

# ## TODO: stupid bank error.  can we use google sheets or some other way convert the currency?

# ep = EasyPeasy() 
# df['converted_value'] = df.apply(lambda x: ep.currency_converter(x['value'], from_currency=x['currency'], to_currency="USD"), axis=1)

In [ ]:
df['value'] = df['value'].astype(int)

In [ ]:
df = df[['year', 'country', 'value', 'gender']]

In [ ]:

df.to_csv("wages.csv", index=False)

In [ ]:
erins = ['New Zealand',
         'Japan',
         'United States',
         'Canada',
         'United Kingdom',
         'Sweden',
         'Norway',
         'Korea',
         'Slovak Republic',
         'Austria',
         'Mexico',
         'Australia',
         'Hungary',
         'Czech Republic',
         'Germany',
         'Finland',
         'Portugal',
         'Israel',
         'Colombia',
         'Belgium',
         'Denmark',
         'Greece',
         'France',
         'Iceland',
         'Costa Rica']

print(len(erins))

erins2 = set(erins)
mycountries = df['country']
print("Num countries in common:",len(erins2.intersection(mycountries))) 